# Welcome to the Critical Minerals Visualizer Notebook

## First we import some python packages we will use

In [1]:
import pandas as pd
import geopandas as gpd
import glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style='ticks', context='talk')

## Below we read in the data

In [2]:
path = r'J:\\Geology\\WSGS\\Projects\\Critical Minerals\\probability maps\\'
allFiles=glob.glob(path+"*.csv")
frame=pd.DataFrame
listed=[]
for file in allFiles:
    df = pd.read_csv(file, index_col=[0])
    listed.append(df)
frame = pd.concat(listed)
crustal_abundance=pd.read_csv(r'J:\Geology\WSGS\Projects\Critical Minerals\crustal_abundance.csv',encoding = "ISO-8859-1")

### Next we read in the spatial component of the data

In [3]:
huc12=gpd.read_file(r'J:\Geology\WSGS\GIS\huc_12.shp') #reads in the hydraulic basin units and adds an area column
huc12= huc12.to_crs({'init': 'epsg:3732'})


counties = gpd.read_file(r'J:\Geology\WSGS\GIS\County_boundaries_polys.shp')
counties = counties.to_crs({'init': 'epsg:3732'})

roads = gpd.read_file(r'J:\Geology\WSGS\GIS\Highway_road.shp')
roads = roads.to_crs({'init': 'epsg:3732'})

frame['HUC_12']=frame.HUC_12.astype(str)
CaptainHucstable = huc12.merge(frame, on='HUC_12' )
pruned = CaptainHucstable[CaptainHucstable['prob']>0]

# Then we can visualize the results
* First you select a formation you are interested in from the drop down menu
* Second, select an element you are interested in from the drop down menu
* The output map is colored by probability for each area, darker red colors indicate a higher probability of finding 5 times the crustal abundance of the mineral you selected in the formation you selected

In [4]:
import ipywidgets as widgets
from ipywidgets import interactive
from ipywidgets import interact
%matplotlib nbagg

formations = sorted(CaptainHucstable['formation'].unique().tolist())
elements = sorted(CaptainHucstable['element'].unique().tolist())
 
def view2(formation='', element='', probability=''):
    
    try:
        fig, ax = plt.subplots()
        counties.plot(ax=ax, alpha=0.8, color='black', edgecolor='white')
        #roads.plot(ax=ax, color='black')

        pruned[(pruned['formation']==formation) & (pruned['element']==element) &
               (pruned['bins_ppm']>=crustal_abundance[element][5]*5) & (
               pruned['prob']>=probability)].plot(ax=ax, 
                    column='prob', cmap='OrRd', legend=True)
    except:
        print('This combination does not exist')
    


w = widgets.Select(options=formations)
v = widgets.Select(options=elements)
u = widgets.FloatSlider(min=0.1,max=1.0,step=0.1,value=0.5)


interact(view2, formation=w, element=v, probability=u)
#interact(view, y=v)

interactive(children=(Select(description='formation', options=('Absaroka Volcanic Supergroup: Hominy Peak Form…

<function __main__.view2>

## This section you can specify the concentration of elements instead of just 5x crustal abundance

In [5]:
import ipywidgets as widgets
from ipywidgets import interactive
from ipywidgets import interact
%matplotlib nbagg

formations = sorted(CaptainHucstable['formation'].unique().tolist())
elements = sorted(CaptainHucstable['element'].unique().tolist())
 
def view(formation='', element='', concentration=''):
    
    try:
        fig, ax = plt.subplots()
        pruned[(pruned['formation']==formation) & (pruned['element']==element) &
               (pruned['bins_ppm']>concentration)].plot(ax=ax, 
                    column='prob', cmap='OrRd', legend=True)
        huc12.plot(ax=ax, alpha=0.1)
    except:
        print('This combination does not exist')
    


w = widgets.Select(options=formations)
v = widgets.Select(options=elements)
u = widgets.FloatText(value=1)
interact(view, formation=w, element=v, concentration=u)
#interact(view, y=v)

interactive(children=(Select(description='formation', options=('Absaroka Volcanic Supergroup: Hominy Peak Form…

<function __main__.view>

In [4]:
import os
import re
from tqdm import tqdm


In [5]:
%matplotlib inline
formations = sorted(CaptainHucstable['formation'].unique().tolist())
elements = sorted(CaptainHucstable['element'].unique().tolist())
 
def view3(formation='', element='', probability=''):
    
    try:
        fig, ax = plt.subplots(figsize=(15, 10))
        counties.plot(ax=ax, alpha=0.8, color='black', edgecolor='white')
        pruned[(pruned['formation']==formation) & (pruned['element']==element) &
               (pruned['bins_ppm']>=crustal_abundance[element][5]) & (
               pruned['prob']>=probability)].plot(ax=ax, 
                    column='prob', cmap='OrRd', legend=True)
        ax.set_title(str(formation) + '\n' +str( element))
        formation = re.sub(':','', formation)
        newpath = ((r'I:/Scratch/Jesse/Minerals and Formations/%s') % (formation))
        if not os.path.exists(newpath): os.makedirs(newpath)
        plt.savefig(r'I:/Scratch/Jesse/Minerals and Formations/'+str(formation)+'/'+str(element)+'.png')
        plt.close()
    except:
        pass
#         print('This combination does not exist')
    


In [ ]:
for i in tqdm(range(len(formations))):
    for j in range(len(elements)):
        view3(formations[i], elements[j],0.5)

  0%|                                                                                          | 0/208 [00:00<?, ?it/s]C:\Users\jesse.pisel\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\ops.py:816: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(x, name)(y)
C:\Users\jesse.pisel\AppData\Local\Continuum\anaconda3\lib\site-packages\matplotlib\pyplot.py:528: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
 70%|████████████████████████████████████████████████████████▏                       | 146/208 [57:38<24:28, 23.69s/it]